# Robert Koch-Institut: COVID-19-Dashboard
 Data from https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4 dashboard using the arcgis REST API.

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime
import re
import pycountry

In [ ]:
# papermill parameters
output_folder = "../output/"

### Get record count, set query parameters

In [ ]:
endpoint = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/ArcGIS/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=false&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson"

# send get request to endpoint
response = requests.get(endpoint + "&returnCountOnly=true")

# check request status is HTTP_OK
assert response.status_code is 200

# load to json
data = json.loads(response.text)

# set params
record_count = data['count']
offset = 0
record_fetch_count = 2000
fields = None
records = []
print(record_count)

### Retrieve data

In [ ]:
while offset < record_count:
    response = requests.get(endpoint + f"&resultRecordCount={record_fetch_count}&resultOffset={offset}")
    
    # check if request status code is HTTP_OK 
    assert response.status_code is 200
    
    # load to json
    data = json.loads(response.text)
    
    # set fields
    if not fields:
        fields = list(map(lambda f: f['name'],data['fields']))
    
    data = list(map(lambda record: record['attributes'].values(), data['features']))
    
    # add to records
    records += data
    
    # set params
    offset += record_fetch_count
    
assert len(records) == record_count
records = dict(zip(range(len(records)), records)) 
df = pd.DataFrame.from_dict(records, orient="index", columns=fields)

### Cleanse data

In [ ]:
# rename columns
column_name_map = {
    "RS": "district_id",
    "county": "county",
    "BL_ID": "state_id",
    "BL": "state",
    "BEZ": "district_type",
    "EWZ": "population",
    "death_rate": "death_rate",
    "cases": "cases",
    "deaths": "deaths",
    "cases_per_100k": "cases_per_100k",
    "cases_per_population": "cases_per_population",
    "cases7_per_100k": "cases7_per_100k",
    "last_update": "last_update",   
}
df.rename(columns=column_name_map, inplace=True)

# map district types
district_type_map = {
    "Landkreis" :"Rural District",
    "Kreis": "Rural District",
    "Kreisfreie Stadt": "District-free City",
    "Stadtkreis": "District-free City"
}

df['district_type'] = df['district_type'].apply(lambda d: district_type_map.get(d))

# set district id as index
df.set_index("district_id")

# select columns by column_name_map
df = df.loc[:, tuple(column_name_map.values())]

# parse last_update
df['last_update'] = df['last_update'].apply(lambda row: pd.to_datetime(row, format="%d.%m.%Y, %H:%M Uhr").tz_localize("Europe/Berlin"))

# set ISO3166_1, ISO3166_2 fields
df['ISO3166_1'] = 'DE'
subdivisions = {k.name: k.code.replace("DE-", "") for k in pycountry.subdivisions.get(country_code="DE")}
df['ISO3166_2'] = df['state'].apply(lambda c: subdivisions.get(c))

In [ ]:
df.dtypes  # leave dataframe dtype as "object" (string as dtype is not supported in current version of pandas)

In [ ]:
df.head()

In [ ]:
df.to_csv(output_folder + "RKI_GER_COVID19_DASHBOARD.csv", index=False)